In [4]:
import pandas as pd
import os
from os.path import expanduser
import re
# establish file data directory
homeDirectory = expanduser("~")
dataDirectory = homeDirectory + os.sep + 'Google Drive/tACS_VWM_ALPHA/data/stim1/'
# read in csv file
saveDirectory = dataDirectory + 's' + str(2) + os.sep + 'runData/'
directoryFiles = os.listdir(saveDirectory)
csv = ''
for file in directoryFiles:
    match = re.match('run' + str(1) + '.*', file, re.M|re.I)
    if match != None:
        csv = saveDirectory + match.group()
df = pd.read_csv(csv)

# create conditions
resps = df['Response'] == 1
noResps = df['Response'] == 0
changes = df['ChangeTrial'] == 1
noChanges = df['ChangeTrial'] == 0
lChanges = df['ChangeCond'] == 1
lNoChanges = df['ChangeCond'] != 1
rChanges = df['ChangeCond'] == 2
rNoChanges = df['ChangeCond'] != 2

In [20]:
df

,trialID,Response,RT,ChangeTrial,nDistractors,nTargets,WFCond,ChangeCond,leftTargs,rightTargs,leftDists,rightDists,changeHemi,Unnamed: 13
0,0,1,0.608519,0,0,4,4,3,2,2,0,0,NaN,NaN
1,1,0,NaN,0,4,4,6,3,2,2,2,2,NaN,NaN
2,2,0,NaN,1,4,4,6,1,2,2,2,2,left,NaN
3,3,1,0.584683,1,4,2,3,2,1,1,2,2,right,NaN
4,4,1,0.384783,0,0,4,4,3,2,2,0,0,NaN,NaN
5,5,0,NaN,0,4,4,6,3,2,2,2,2,NaN,NaN
6,6,0,NaN,0,4,4,6,3,2,2,2,2,NaN,NaN
7,7,0,NaN,1,2,2,2,2,1,1,1,1,right,NaN
8,8,1,0.650805,1,0,4,4,2,2,2,0,0,right,NaN
9,9,1,0.634981,1,4,2,3,1,1,1,2,2,left,NaN
